In [2]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import regularizers
import yfinance as yf

# Função para carregar os dados ou obter novos dados
def load_or_get_data(ticker, start_date, end_date):
    try:
        # Tenta carregar os dados salvos
        train_data = np.load(f"{ticker}_train_data.npy")
        test_data = np.load(f"{ticker}_test_data.npy")
        print("Dados carregados com sucesso.")
    except FileNotFoundError:
        # Se os dados não existirem, obtém novos dados
        print("Dados não encontrados. Obtendo novos dados...")
        data = yf.download(ticker, start=start_date, end=end_date)
        scaler = MinMaxScaler()
        data_scaled = scaler.fit_transform(data[['Close', 'Volume']].values)
        train_size = int(len(data_scaled) * 0.8)
        train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]
        np.save(f"{ticker}_train_data.npy", train_data)
        np.save(f"{ticker}_test_data.npy", test_data)
    return train_data, test_data

# Solicita ao usuário inserir o símbolo do ativo
ticker = input(" ATENÇÃO= Este Robô é um exemplo para testes e para fins didáticos utilize com sabedoria. Os ativos utilizam preços de fechamento e devem ser usados para previsões de longo prazo. Insira o símbolo do ativo em caixa alta (ex: BOVA11.SA, BTC-USD, AAPL): ")

# Solicita ao usuário inserir a data de início do período de treinamento
start_date = input("Insira a data de início do período de treinamento (no formato AAAA-MM-DD): ")

# Solicita ao usuário inserir a data de fim do período de treinamento
end_date = input("Insira a data de fim do período de treinamento (no formato AAAA-MM-DD): ")

# Carrega ou obtém os dados com base nas datas fornecidas pelo usuário
train_data, test_data = load_or_get_data(ticker, start_date, end_date)

# Preparando os dados para a rede neural
def create_dataset(dataset, time_steps=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_steps - 1):
        a = dataset[i:(i + time_steps), :]
        dataX.append(a)
        dataY.append(dataset[i + time_steps, 0])  # Usaremos apenas o preço para o Y
    return np.array(dataX), np.array(dataY)

time_steps = 50  # Número de períodos anteriores a serem usados como entrada (uma semana)
train_X, train_Y = create_dataset(train_data, time_steps)
test_X, test_Y = create_dataset(test_data, time_steps)

# Construindo a rede neural com mais neurônios e camadas de regularização
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(time_steps, 2)),  # Dois inputs: preço e volume
    tf.keras.layers.LSTM(150, return_sequences=True, kernel_regularizer=regularizers.l2(0.001)), # Regularização L2
    tf.keras.layers.Dropout(0.2),  # Dropout para evitar overfitting
    tf.keras.layers.LSTM(150, kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Treinando a rede neural com mais épocas
model.fit(train_X, train_Y, epochs=50, batch_size=64, verbose=1)

# Fazendo previsões
test_predictions = model.predict(test_X)

# Comparando a previsão com o último preço real
last_real_price = test_data[-1, 0]
predicted_price = test_predictions[-1, 0]

# Determinando se o ativo irá subir, cair ou permanecer neutro
if predicted_price > last_real_price:
    prediction_text = "O ATIVO IRÁ SUBIR! (BULL MARKET)"
elif predicted_price < last_real_price:
    prediction_text = "O ATIVO IRÁ CAIR! (BEAR MARKET)"
else:
    prediction_text = "O ATIVO IRÁ PERMANECER NEUTRO!"

print(prediction_text)


Insira o símbolo do ativo em caixa alta (ex: BOVA11.SA): btc-usd
Insira a data de início do período de treinamento (no formato YYYY-MM-DD): 2023-01-01
Insira a data de fim do período de treinamento (no formato YYYY-MM-DD): 2024-03-20
Dados não encontrados. Obtendo novos dados...


[*********************100%%**********************]  1 of 1 completed


Epoch 1/50
5/5 [==============================] - 5s 226ms/step - loss: 0.3728
Epoch 2/50
5/5 [==============================] - 1s 228ms/step - loss: 0.3186
Epoch 3/50
5/5 [==============================] - 2s 390ms/step - loss: 0.2822
Epoch 4/50
5/5 [==============================] - 2s 292ms/step - loss: 0.2499
Epoch 5/50
5/5 [==============================] - 1s 216ms/step - loss: 0.2209
Epoch 6/50
5/5 [==============================] - 1s 227ms/step - loss: 0.1945
Epoch 7/50
5/5 [==============================] - 1s 220ms/step - loss: 0.1717
Epoch 8/50
5/5 [==============================] - 1s 216ms/step - loss: 0.1513
Epoch 9/50
5/5 [==============================] - 1s 218ms/step - loss: 0.1336
Epoch 10/50
5/5 [==============================] - 1s 217ms/step - loss: 0.1180
Epoch 11/50
5/5 [==============================] - 1s 220ms/step - loss: 0.1045
Epoch 12/50
5/5 [==============================] - 1s 223ms/step - loss: 0.0925
Epoch 13/50
5/5 [==============================] 